In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
FILE = '~/GitHub/ci-analysis/travistorrent_11_1_2017.csv.gz'

In [ ]:
dataset = pd.read_csv(FILE, sep=',', engine='c',usecols=['gh_team_size',
                                                         'tr_build_id',
                                                         'gh_project_name',
                                                         'tr_build_id',
                                                         'gh_project_name',
                                                         'gh_is_pr',
                                                         'gh_pr_created_at',
                                                         'gh_lang',
                                                         'git_branch',
                                                         'gh_num_commits_in_push',
                                                         'git_prev_commit_resolution_status',
                                                         'tr_prev_build',
                                                         'gh_first_commit_created_at',
                                                         'gh_team_size',         
                                                         'gh_pushed_at',
                                                         'gh_sloc',
                                                         'tr_log_bool_tests_failed',
                                                         'tr_log_setup_time',
                                                         'tr_log_buildduration',
                                                         'tr_status'],dtype={ 'tr_build_id': np.float32,
                                                                                            'gh_project_name': 'category',
                                                                                            'gh_is_pr': np.bool,
                                                                                            'gh_pr_created_at': 'category',
                                                                                            'gh_lang': 'category',
                                                                                            'git_branch': 'category',
                                                                                            'gh_num_commits_in_push': np.float32,
                                                                                            'git_prev_commit_resolution_status': 'category',
                                                                                            'tr_prev_build': np.float32,
                                                                                            'gh_first_commit_created_at': 'category',
                                                                                            'gh_team_size': np.int32,
                                                                                            'gh_pushed_at': 'category',                                                                                           
                                                                                            'gh_sloc': np.int32}, encoding='utf-8')


In [ ]:
dataset = dataset.loc[dataset['git_branch'] == 'master'] #filter only projects in the master
dataset = dataset[dataset.gh_lang != 'javascript'] #remove projects java script
dataset = dataset.dropna(subset=['gh_num_commits_in_push']) #remove NaN in the column specified
dataset['gh_pushed_at'] = pd.to_datetime(dataset.gh_pushed_at) #transforms for datetime
dataset = dataset.drop_duplicates(['tr_build_id']) #remove duplicates

In [1]:
### Filters the Java language and returns a list with the name of the projects

In [ ]:
df= dataset.loc[dataset['gh_lang'] == 'java']
list_name = df.gh_project_name.unique().tolist()

In [ ]:
### Creates 2 filters for passing and non-passing builds

In [ ]:
passed = df.loc[df['tr_status'] == 'passed']
failed = df.loc[df['tr_status'] == 'failed']

# print('Projetos Java')
# print('Total de builds que passaram {}'.format(passed.size))
# print('Total de builds que NÃO passaram {}'.format(failed.size))

In [ ]:
### Creates a list with the amount builds passed and failed

In [ ]:
passed = []
failed = []
for i in range(len(list_name)):
    project = df.loc[df['gh_project_name'] == list_name[i]] 
    _passed = project.loc[project['tr_status'] == 'passed']
    _failed = project.loc[project['tr_status'] == 'failed']
    
    passed.append(_passed.size)
    failed.append(_failed.size)

In [ ]:
### Status Build

In [ ]:
def status_build(project_name):
    #filtra o dataset pelo nome do projeto e ordena as datas do dataset
    df = dataset.loc[dataset['gh_project_name'] == project_name]
    
    df = df.sort_values(by='gh_pushed_at', ascending=False)#ordena da data mais atual para mais antiga
    
    df['gh_pushed_at'] = pd.to_datetime(df.gh_pushed_at).dt.date#le a coluna com apenas os dias
    
    date_line = df.gh_pushed_at # cria uma Serie com as datas
    status_build = df.tr_status.tolist() # cria uma lista com os status da build

    
    '''
    cont = 0

    for i in range(len(status_build)):

        if('failed' == status_build[i]):


            print(f'failed: {i} | time: {cont} | {time[i]}')
            cont += 1

        elif('passed' == status_build[i]):

            print(f'passed: {i} | time: {cont} | {time[i]}')
    '''
    
    build_red = []
    sett = 0
    
    for i in range(len(status_build)):

        if("failed" == status_build[i]):
            if(sett == 0):
                index_fail = i
                sett = 1
        if(('passed' == status_build[i]) and (sett == 1)):
                
                #print(f"{project_name} | build: fail:{index_fail} passed:{i} | linha: {i}: days: {date_line.iloc[i]} - {date_line.iloc[index_fail]} | day: {date_line.iloc[index_fail] - date_line.iloc[i]}")
                build_red.append(date_line.iloc[index_fail] - date_line.iloc[i]) # subtrai o dia que a build nao passou ate o dia que passa
                sett = 0

    return(build_red) # retorna uma lista com datas de quando deu fail ate pasar

In [ ]:
### Status Build for all projects

In [ ]:
list_name = dataset.gh_project_name.unique().tolist() 

#print(f"quantidade total de projetos {len(list_name)}")

list_days_broken = []
for i in range(len(list_name)):
    list_days_broken.append(status_build(list_name[i]))

days_broken = []
for j in range(len(list_days_broken)):
    days_broken = days_broken + list_days_broken[j]
    


days = pd.DataFrame(days_broken, columns=['date']) # cria uma

#print(days.date.dt.days.describe())


count = 0 
metric = days.date.dt.days.describe()[6] # 3 quartil 

#percorre o vetor quando ele encontra um que é equivalente ao if da break e vai para o proximo vetor que no caso seria outro projeto
for k in range(len(list_days_broken)):
    validate = 0 
    for j in range(len(list_days_broken[k])):
        if(metric <= list_days_broken[k][j].days):
            count+=1
            break
            #se uma metrica está sendo contada mais de uma ves quando ele encontar deve passar pra proxima lista e não fica na mesma 

#print(f"Quantidade de projetos com builds quebradas é de {count} tendo uma duração de {metric} dias")
   